In [ ]:
import pandas as pd
from math import radians, cos, sin, sqrt, atan2

# 거리 계산 함수 (Haversine 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0 # 지구의 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# 데이터 불러오기
df_districts = pd.read_excel('북구 행정동 중심좌표.xlsx')
transport_coords = pd.read_excel('북구 버스정류장+지하철역 위치.xlsx')

# 위도와 경도를 float 형식으로 변환
df_districts['위도'] = df_districts['위도'].astype(float)
df_districts['경도'] = df_districts['경도'].astype(float)
transport_coords['위도'] = transport_coords['위도'].astype(float)
transport_coords['경도'] = transport_coords['경도'].astype(float)

# 접근성 점수 계산 함수
def calculate_accessibility_score(lat, lon, transport_coords, radius=0.5):
    scores = []
    for _, row in transport_coords.iterrows():
        lat_t = row['위도']
        lon_t = row['경도']
        dist = haversine(lat, lon, lat_t, lon_t)
        if dist <= radius:
            score = 1 / (dist + 0.1)
            scores.append(score)
    return sum(scores)

# 각 행정구역에 대해 접근성 지수 계산
df_districts['접근성지수'] = df_districts.apply(
    lambda row: calculate_accessibility_score(row['위도'], row['경도'], transport_coords),
    axis=1
)

print(df_districts[['행정구역', '접근성지수']])


     행정구역      접근성지수
0     고성동  39.716178
1     칠성동  57.312698
2     침산동  27.364019
3     산격동  35.996280
4     대현동  16.450682
5     복현동  26.774431
6     검단동  21.046215
7   무태조야동  44.089187
8     관문동  31.618313
9     태전동  47.900047
10    구암동  10.767114
11    관음동  16.260187
12    읍내동   0.000000
13    동천동  37.812738
14    노원동  24.112044
15    국우동   3.673619


In [ ]:
df_districts.to_excel('북구 행정구역 접근성지수.xlsx', index=False)